In [1]:
import os, shlex, subprocess

repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = r"""
python train.py
--dataroot ./datasets/coraline_p2p_v2
--name coraline_p2p_512_e50_v2
--model pix2pix
--dataset_mode aligned
--direction AtoB
--input_nc 3
--output_nc 3
--netG unet_256
--lambda_L1 100
--gan_mode lsgan
--preprocess resize_and_crop
--load_size 544
--crop_size 512
--batch_size 1
--gpu_ids 0
--n_epochs 20
--n_epochs_decay 30
--save_epoch_freq 5
--print_freq 50
--display_freq 200
"""
proc = subprocess.run(shlex.split(" ".join(cmd.split())), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\en

In [8]:
import os, subprocess

os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TORCH_COMPILE_DISABLE"]  = "1"
os.environ["TORCHDYNAMO_DISABLE"]    = "1"

# === CONFIG ===
repo_dir    = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
dataroot    = r".\datasets\coraline_p2p_v2"
name        = "coraline_p2p_512_e50_v2"
epoch       = "50"
gpu_ids     = "0"
num_test    = "100000"
results_dir = r".\results"

preprocess  = "none"
load_size   = "512"
crop_size   = "512"

cmd = [
    "python", "test.py",
    "--dataroot", dataroot,
    "--name", name,
    "--model", "pix2pix",
    "--dataset_mode", "aligned",
    "--direction", "AtoB",
    "--epoch", epoch,
    "--gpu_ids", gpu_ids,
    "--num_test", num_test,
    "--results_dir", results_dir,
    "--netG", "unet_256",
    "--input_nc", "3",
    "--output_nc", "3",
    "--preprocess", preprocess,
    "--load_size", load_size,
    "--crop_size", crop_size,
    "--no_flip",
    "--serial_batches",
    "--eval",
]

os.chdir(repo_dir)
print("CWD:", os.getcwd())
print("Running:\n", " ".join(cmd))

proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)

run_tag = f"test_{epoch}"
images_dir = os.path.abspath(os.path.join(results_dir, name, run_tag, "images"))
print("Results folder:", images_dir)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
Running:
 python test.py --dataroot .\datasets\coraline_p2p_v2 --name coraline_p2p_512_e50_v2 --model pix2pix --dataset_mode aligned --direction AtoB --epoch 50 --gpu_ids 0 --num_test 100000 --results_dir .\results --netG unet_256 --input_nc 3 --output_nc 3 --preprocess none --load_size 512 --crop_size 512 --no_flip --serial_batches --eval
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 512                           	[default: 256]
                 dataroot: .\datasets\coraline_p2p_v2   

In [7]:
import os, glob, re, cv2

input_dir   = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\results\coraline_p2p_v3_512\test_50\test video" 
output_mp4  = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\results\coraline_p2p_v3_512\test_50\test videoepoch50videovideoskipped.mp4"
fps         = 12
extensions  = ("*.png","*.jpg","*.jpeg","*.bmp","*.webp")

def natural_key(s):
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r'(\d+)', s)]

files = []
for ext in extensions:
    files.extend(glob.glob(os.path.join(input_dir, ext)))
files.sort(key=natural_key)

if not files:
    raise SystemExit("No images found.")

first = cv2.imread(files[0], cv2.IMREAD_COLOR)
if first is None:
    raise SystemExit(f"Cannot read: {files[0]}")
h, w = first.shape[:2]

w += w % 2
h += h % 2

fourcc = cv2.VideoWriter_fourcc(*"mp4v") 
out = cv2.VideoWriter(output_mp4, fourcc, fps, (w, h))
if not out.isOpened():
    raise SystemExit("Failed to open VideoWriter. Try a different fourcc (e.g., 'avc1').")

for i, f in enumerate(files, 1):
    img = cv2.imread(f, cv2.IMREAD_COLOR)
    if img is None:
        print(f"Skip unreadable: {f}")
        continue
    if (img.shape[1], img.shape[0]) != (w, h):
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_AREA)
    out.write(img)
    if i % 50 == 0 or i == len(files):
        print(f"{i}/{len(files)} frames written")

out.release()
print("Saved:", os.path.abspath(output_mp4))


20/20 frames written
Saved: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\results\coraline_p2p_v3_512\test_50\test videoepoch50videovideoskipped.mp4


In [6]:
import os, re, glob, shutil

# ===== CONFIG =====
in_dir  = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\results\coraline_p2p_v3_512\test_50\test video"
out_dir = in_dir + "_every_other"  
keep_every = 2                    
start_offset = 0                    
extensions = ("*.png","*.jpg","*.jpeg","*.bmp","*.webp")
DELETE_IN_PLACE = False            

def nkey(s): return [int(t) if t.isdigit() else t.lower() for t in re.split(r'(\d+)', os.path.basename(s))]

files = []
for ext in extensions:
    files += glob.glob(os.path.join(in_dir, ext))
files.sort(key=nkey)

print(f"Found {len(files)} frames in {in_dir}")
if not files:
    raise SystemExit("No frames found.")

if not DELETE_IN_PLACE:
    os.makedirs(out_dir, exist_ok=True)

kept = 0
removed = 0
for i, src in enumerate(files):
    keep = (i - start_offset) % keep_every == 0
    if keep:
        if DELETE_IN_PLACE:
            pass
        else:
            dst = os.path.join(out_dir, os.path.basename(src))
            shutil.copy2(src, dst)
        kept += 1
    else:
        if DELETE_IN_PLACE:
            os.remove(src)
        removed += 1

print(f"Kept: {kept}  Removed/Skipped: {removed}")
if not DELETE_IN_PLACE:
    print("Output ->", os.path.abspath(out_dir))


Found 20 frames in C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\results\coraline_p2p_v3_512\test_50\test video
Kept: 10  Removed/Skipped: 10
Output -> C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\results\coraline_p2p_v3_512\test_50\test video_every_other


In [4]:
import os
from PIL import Image

input_folder = r"C:\Users\yalda\OneDrive\Desktop\Tpose Coraline Stylle\amy fight"

output_folder = os.path.join(input_folder, "AB_pairs")
os.makedirs(output_folder, exist_ok=True)

target_size = (512, 512)

for filename in os.listdir(input_folder):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(input_folder, filename)

        img = Image.open(img_path).convert("RGB")
        img_resized = img.resize(target_size, Image.LANCZOS)

        ab_img = Image.new("RGB", (target_size[0] * 2, target_size[1]))
        ab_img.paste(img_resized, (0, 0))  
        ab_img.paste(img_resized, (target_size[0], 0))  

        save_path = os.path.join(output_folder, filename)
        ab_img.save(save_path)

print(f"Done! AB pairs saved in: {output_folder}")


Done! AB pairs saved in: C:\Users\yalda\OneDrive\Desktop\Tpose Coraline Stylle\electrocute normal\AB_pairs
